In [ ]:
import pandas as pd
from geolite2 import geolite2
import re

## 1) Извлечь как можно больше информации из ip-адреса и user agent-а и записать в отдельный csv файл

In [ ]:
data = pd.read_csv('test.csv', sep=";", header=0)

###### Подготовливаем ip адреса для проверки  через geolite2

In [ ]:
data_for_geolite = data.ip.str.replace(r"['\s([)\&\]]","")

In [ ]:
data_for_geolite = data_for_geolite.str.replace(r",.*","")

###### Убираем предидущее столбец с ip и объединяем данные

In [ ]:
data.drop("ip", axis=1, inplace=True)

In [ ]:
data_for_geolite = pd.concat((data_for_geolite, data), axis=1)

###### Создаем функцию для проверки и проверяем ip адреса

In [ ]:
def f(item):
    match = geolite2.reader()
    return match.get(item)

In [ ]:
data_from_geolite = data_for_geolite.ip.apply(f)

In [ ]:
data_from_geolite = pd.concat((data_from_geolite, data_for_geolite), axis=1)

In [ ]:
labels = ["ip_info", "ip", "device_id", "login","user_agent"]

In [ ]:
data_from_geolite.columns = labels

###### Извлекаем данные из user_agent с помощью регулярных выражений и сохраняем файл

In [ ]:
data_user_agent = data_from_geolite["user_agent"]

In [ ]:
data_os = data_user_agent.replace("^Mozilla/5.0\s\(|\).*", "", regex=True)

In [ ]:
data_mob = data_user_agent.str.contains(".obile", regex=True)

In [ ]:
re_browsers = "(\S*$)|(Chr.*)"

In [ ]:
data_browsers = data_user_agent.str.extract(re_browsers)

In [ ]:
data_browsers = data_browsers[0].fillna(data_browsers[1])

In [ ]:
data_to_csv = pd.concat((data_from_geolite, data_os, data_browsers, data_mob ), axis=1)

In [ ]:
labels2 = ["ip_info", "ip", "device_id", "login","user_agent", "os", "browser", "mobile"]

In [ ]:
data_to_csv.columns = labels2

In [ ]:
data_to_csv.to_csv("results.csv")

# 2) Показать, какие идентификаторы устройства принадлежат одному и тому же реальному устройству и почему?

###### Создаем датафрейм с идентификаторами и информации из user_agent и группируем

In [ ]:
data_devid_usagent = data_to_csv[["user_agent", "device_id"]]

In [ ]:
for _, x in data_devid_usagent.groupby(["user_agent"]): print (60*"-","\n\n", x)

# 3) Найти связи между устройствами и логинами

###### Создаем датафрейм с нужными колонками и группируем по логину

In [ ]:
data_login = data_to_csv[["login","device_id",  "os", "browser"]]

In [ ]:
for _, x in data_login.groupby(["login"]): print (60*"-","\n\n", x)

###### Просматриваем наиболее используемый логин и устройства на которых он использовался

In [ ]:
top_logins = data_to_csv["login"].value_counts()

In [ ]:
top_logins.head(10)

In [ ]:
the_top_login = data_to_csv[data_to_csv['login'] == "ed9b3e652da241191a4d2f52d6c89d84295a49d3"]

In [ ]:
the_top_login["user_agent"].value_counts()